In [1]:
# Useful additional packages
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from math import pi
import qiskit
import random
import math

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import state_fidelity
from qiskit import BasicAer
from qiskit.visualization import plot_state_city
import pennylane as qml

backend = BasicAer.get_backend('qasm_simulator')
backendState = BasicAer.get_backend('statevector_simulator')

### _Intro: Making the circuit which fits the criteria_


In [2]:
q = QuantumRegister(2)
c = ClassicalRegister(2)
circ = QuantumCircuit(q,c)

circ.h(0)
circ.x(1)
circ.cx(0,1)

circ.measure(q,c)

jobShots = execute(circ, backend, shots=1024)
jobShots.result().get_counts(circ)

{'10': 522, '01': 502}

###  _Using functions which fit the desired state and applying gradient descent_

In [3]:
def steepestDescentTheta1(init, tol, nmax, scale):
    theta = init
    n=0
    eps = 0
    while n<nmax or eps>tol:
        yhat = -math.sin(theta)
        dydt = -math.cos(theta)
        
        theta_next = theta - scale*dydt
        
        eps = ((theta_next-theta)**2)

        theta = theta_next
        n=n+1
        
    return theta
    

In [4]:
def steepestDescentTheta2(init, tol, nmax, scale):
    theta = init
    n=0
    eps = 0
    while n<nmax or eps>tol:
        yhat = math.cos(theta)
        dydt = -math.sin(theta)
        
        theta_next = theta - scale*dydt
        
        eps = ((theta_next-theta)**2)

        theta = theta_next
        n=n+1
        
    return theta

In [5]:
def steepestDescentBoth(init, tol, nmax, scale):
    theta1 = init[0]
    theta2 = init[1]
    n=0
    eps1 = 1
    eps2 = 1
    while n<nmax:
        
        yhat = -math.sin(theta1) + math.cos(theta2) 
        dydt1 = -math.cos(theta1)
        dydt2 = -math.sin(theta2) 
        
        theta1_next = theta1 - dydt1
        theta2_next = theta2 - dydt2
        
        
        if (eps1 < tol) :
            eps2 = (theta2_next-theta2)**2 
            theta2 = theta2_next
            
        if (eps2 < tol): 
            eps1 = (theta1_next-theta1)**2 
            theta1 = theta1_next
            
        elif (eps1>=tol) and (eps2>=tol):
            eps1 = (theta1_next-theta1)**2 
            eps2 = (theta2_next-theta2)**2
            
            theta1 = theta1_next
            theta2 = theta2_next
        
        n=n+1
        
    print(n)
    return [theta1, theta2]

In [6]:

qr2 = QuantumRegister(2)
c2 = ClassicalRegister(2)
qc2 = QuantumCircuit(qr2,c2)

theta1_init = random.random()*2*pi
theta2_init = random.random()*2*pi


#th1 = steepestDescentTheta1(theta1_init, 1, 40000, 1)
#th2 = steepestDescentTheta2(theta2_init, 1, 40000, 1)

[th1, th2] = steepestDescentBoth(np.array([theta1_init, theta2_init]), 1e-6 ,10000, 1)


print("Theta 1 and Theta 2: ", th1, th2)

10000
Theta 1 and Theta 2:  1.5707963267948966 3.141592653589793


In [7]:
qc2.ry(th1, 0)
qc2.ry(th2, 1)
qc2.cx(0,1)


job2 = execute(qc2, backendState)
fullState = job2.result().get_statevector(qc2, decimals=3)
print(fullState)

[0.        +0.j 0.70710678+0.j 0.70710678+0.j 0.        +0.j]


In [8]:
qc2.measure(qr2,c2)

In [9]:
jobShots = execute(qc2, backend, shots=1)
jobShots.result().get_counts(qc2)

{'10': 1}

In [10]:
jobShots = execute(qc2, backend, shots=100)
jobShots.result().get_counts(qc2)

{'10': 53, '01': 47}

In [11]:
jobShots = execute(qc2, backend, shots=1000)
jobShots.result().get_counts(qc2)

{'01': 462, '10': 538}

In [ ]:
#This was a nice exercise in circuits, I enjoyed matching a real 2D function to the desired state and learned how
# to optimize on single parameters to achieve it. I could have made this harder by using RX gates as well
# but I believe RY gates are just a more elegant method. 

#To make the state |01>-|10>, we would keep the second initial state at |0> 





#Here is a pattern I found in the state described (not grad des):

'''
err1 = abs( 0 - fullState[0]) 
err2 = abs( 1/np.sqrt(2) - fullState[1]) 
err3 = abs( 1/np.sqrt(2) - fullState[2]) 
err4 = abs( 0 - fullState[3]) 
f = (err1 + err2 + err3 + err4)**2

stepsize = 0.001
while f>0.001 :
    qc.data.pop()
    qc.data.pop()
    qc.data.pop()
    
    
    #####Pattern
    #if prob(00)=0 and prob(11)=0 second qubit is correct
    if math.isclose(fullState[0], 0.0 , abs_tol=1e-3) and math.isclose(fullState[3], 0.0, abs_tol=1e-3):    
        th1 = th1 + stepsize
    
    
    #if prob 00=11 and prob 10=01, first qubit is correct
    if math.isclose(fullState[0],fullState[3], rel_tol=1e-3) and math.isclose(fullState[1],fullState[2], rel_tol=1e-3):     
        th2 = th2 + stepsize
    #####
        
    else:
        th1 = th1 + stepsize
        th2 = th2 + stepsize
        
    qc.ry(th1, 0)
    qc.ry(th2, 1)
    qc.cx(0,1)

    
    job2 = execute(qc, backendState)
    fullState = job2.result().get_statevector(qc, decimals=3)

    err1 = abs( 0 - fullState[0]) 
    err2 = abs( 1/np.sqrt(2) - fullState[1]) 
    err3 = abs( 1/np.sqrt(2) - fullState[2]) 
    err4 = abs( 0 - fullState[3])  
    
    f = (err1 + err2 + err3 + err4)**2
        
print(th1, th2)
'''



'''
Also, I could not get this code working as I don't know PennyLane well, and was running out of time. 


import pennylane as qml
from pennylane import expval
from pennylane.optimize import GradientDescentOptimizer

dev = qml.device('default.qubit', wires=2)

@qml.qnode(dev)

def qfunc(th):
    qml.RY(th[0], wires=0)
    qml.RX(th[1], wires=0)
    qml.RY(th[2], wires=1)
    qml.RY(th[3], wires=1)
    qml.CNOT(wires=[0,1])
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

opt = GradientDescentOptimizer(0.1)

th = [0.001, 0.001, 0.001, 0.001]
for it in range(100):
    th = opt.step(qfunc, th)
    print("Step {}: cost:  {}".format(it, qfunc(th)))

'''

